Import libraries

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.activations import relu, selu
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

Load dataset

In [ ]:
# Step 1: Load the heart.csv dataset
data = pd.read_csv('heart.csv')

# Assuming the last column is the target variable
X = data.drop(columns=['target'])
y = data['target']

Split the dataset

In [ ]:
# Step 2: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Scale the features

In [ ]:
# Step 3: Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Define the hyperparameter grid

In [ ]:
# Step 4: Define the hyperparameters as shown in the image
experiments = [
    {'learning_rate': 0.001, 'batch_size': 32, 'activation_function': 'relu', 'dropout_rate': 0.2},
    {'learning_rate': 0.001, 'batch_size': 32, 'activation_function': 'relu', 'dropout_rate': 0.5},
    {'learning_rate': 0.001, 'batch_size': 32, 'activation_function': 'selu', 'dropout_rate': 0.2},
    {'learning_rate': 0.001, 'batch_size': 32, 'activation_function': 'selu', 'dropout_rate': 0.5},
    {'learning_rate': 0.001, 'batch_size': 64, 'activation_function': 'relu', 'dropout_rate': 0.2},
    {'learning_rate': 0.001, 'batch_size': 64, 'activation_function': 'relu', 'dropout_rate': 0.5},
    {'learning_rate': 0.001, 'batch_size': 64, 'activation_function': 'selu', 'dropout_rate': 0.2},
    {'learning_rate': 0.001, 'batch_size': 64, 'activation_function': 'selu', 'dropout_rate': 0.5},
    {'learning_rate': 0.0001, 'batch_size': 32, 'activation_function': 'relu', 'dropout_rate': 0.2},
    {'learning_rate': 0.0001, 'batch_size': 32, 'activation_function': 'relu', 'dropout_rate': 0.5},
    {'learning_rate': 0.0001, 'batch_size': 32, 'activation_function': 'selu', 'dropout_rate': 0.2},
    {'learning_rate': 0.0001, 'batch_size': 32, 'activation_function': 'selu', 'dropout_rate': 0.5},
    {'learning_rate': 0.0001, 'batch_size': 64, 'activation_function': 'relu', 'dropout_rate': 0.2},
    {'learning_rate': 0.0001, 'batch_size': 64, 'activation_function': 'relu', 'dropout_rate': 0.5},
    {'learning_rate': 0.0001, 'batch_size': 64, 'activation_function': 'selu', 'dropout_rate': 0.2},
]

Create model

In [ ]:
# Step 5: Function to create and compile the model
def create_model(learning_rate, activation_function, dropout_rate):
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation=activation_function))
    model.add(Dropout(dropout_rate))
    model.add(Dense(64, activation=activation_function))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

Train the model

In [ ]:
# Step 6: Iterate through the experiments and train the model
results = []
for experiment in experiments:
    model = create_model(
        learning_rate=experiment['learning_rate'],
        activation_function=experiment['activation_function'],
        dropout_rate=experiment['dropout_rate']
    )
    model.fit(X_train, y_train, epochs=50, batch_size=experiment['batch_size'], verbose=0)
    _, accuracy = model.evaluate(X_test, y_test, verbose=0)
    results.append({
        'experiment': experiment,
        'accuracy': accuracy
    })

Results

In [ ]:
# Step 7: Display the results
results_df = pd.DataFrame(results)
pd.set_option('display.precision', 10)
print(results_df)

                                           experiment      accuracy
0   {'learning_rate': 0.001, 'batch_size': 32, 'ac...  0.8032786846
1   {'learning_rate': 0.001, 'batch_size': 32, 'ac...  0.8360655904
2   {'learning_rate': 0.001, 'batch_size': 32, 'ac...  0.8688524365
3   {'learning_rate': 0.001, 'batch_size': 32, 'ac...  0.8360655904
4   {'learning_rate': 0.001, 'batch_size': 64, 'ac...  0.8852459192
5   {'learning_rate': 0.001, 'batch_size': 64, 'ac...  0.9016393423
6   {'learning_rate': 0.001, 'batch_size': 64, 'ac...  0.8524590135
7   {'learning_rate': 0.001, 'batch_size': 64, 'ac...  0.8360655904
8   {'learning_rate': 0.0001, 'batch_size': 32, 'a...  0.8688524365
9   {'learning_rate': 0.0001, 'batch_size': 32, 'a...  0.8524590135
10  {'learning_rate': 0.0001, 'batch_size': 32, 'a...  0.8524590135
11  {'learning_rate': 0.0001, 'batch_size': 32, 'a...  0.8688524365
12  {'learning_rate': 0.0001, 'batch_size': 64, 'a...  0.8524590135
13  {'learning_rate': 0.0001, 'batch_size': 64, 

Hyperparameter Tuning Using GridSearch

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator

# Wrapper class for model creation function
class KerasGridSearch(BaseEstimator):
    def __init__(self, learning_rate=0.001, activation_function='relu', dropout_rate=0.2, batch_size=32):
        self.learning_rate = learning_rate
        self.activation_function = activation_function
        self.dropout_rate = dropout_rate
        self.batch_size = batch_size
        self.model = self.create_model()

    def create_model(self):
        model = Sequential()
        model.add(Dense(64, input_dim=X_train.shape[1], activation=self.activation_function))
        model.add(Dropout(self.dropout_rate))
        model.add(Dense(64, activation=self.activation_function))
        model.add(Dropout(self.dropout_rate))
        model.add(Dense(1, activation='sigmoid'))
        optimizer = Adam(learning_rate=self.learning_rate)
        model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
        return model

    def fit(self, X, y):
        self.model.fit(X, y, epochs=50, batch_size=self.batch_size, verbose=0)
        return self

    def predict(self, X):
        return (self.model.predict(X) > 0.5).astype("int32")

    def score(self, X, y):
        y_pred = self.predict(X)
        return accuracy_score(y, y_pred)

# Create an instance of the wrapper class
model = KerasGridSearch()

# Define the hyperparameter grid for GridSearchCV
param_grid = {
    'learning_rate': [0.001, 0.0001],
    'activation_function': ['relu', 'selu'],
    'dropout_rate': [0.2, 0.5],
    'batch_size': [32, 64]
}

# Create GridSearchCV object
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)

# Fit GridSearchCV to training data
grid_search.fit(X_train, y_train)

3/3 [==============================] - 0s 6ms/step


3/3 [==============================] - 0s 5ms/step


3/3 [==============================] - 0s 5ms/step


GridSearchCV(cv=3, estimator=KerasGridSearch(),
             param_grid={'activation_function': ['relu', 'selu'],
                         'batch_size': [32, 64], 'dropout_rate': [0.2, 0.5],
                         'learning_rate': [0.001, 0.0001]})

In [ ]:
print("Best Hyperparameter: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

Best Hyperparameter:  {'activation_function': 'relu', 'batch_size': 64, 'dropout_rate': 0.2, 'learning_rate': 0.001}
Best score:  0.8140946502057614


HYPERPARAMETER OPTIMIZATION RANDOM SEARCH

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.base import BaseEstimator
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import accuracy_score
import numpy as np

# Wrapper class for model creation function
class KerasRandomSearch(BaseEstimator):
    def __init__(self, learning_rate=0.001, activation_function='relu', dropout_rate=0.2, batch_size=32):
        self.learning_rate = learning_rate
        self.activation_function = activation_function
        self.dropout_rate = dropout_rate
        self.batch_size = batch_size
        self.model = self.create_model()

    def create_model(self):
        model = Sequential()
        model.add(Dense(64, input_dim=X_train.shape[1], activation=self.activation_function))
        model.add(Dropout(self.dropout_rate))
        model.add(Dense(64, activation=self.activation_function))
        model.add(Dropout(self.dropout_rate))
        model.add(Dense(1, activation='sigmoid'))
        optimizer = Adam(learning_rate=self.learning_rate)
        model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
        return model

    def fit(self, X, y):
        self.model.fit(X, y, epochs=50, batch_size=self.batch_size, verbose=0)
        return self

    def predict(self, X):
        return (self.model.predict(X) > 0.5).astype("int32")

    def score(self, X, y):
        y_pred = self.predict(X)
        return accuracy_score(y, y_pred)

# Create an instance of the wrapper class
model = KerasRandomSearch()

# Define the hyperparameter space for RandomizedSearchCV
param_dist = {
    'learning_rate': [0.001, 0.0001],
    'activation_function': ['relu', 'selu'],
    'dropout_rate': [0.2, 0.5],
    'batch_size': [32, 64]
}

# Create RandomizedSearchCV object
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=10, cv=3, verbose=2, random_state=42)

# Fit RandomizedSearchCV to training data
random_search.fit(X_train, y_train)


Fitting 3 folds for each of 10 candidates, totalling 30 fits
3/3 [==============================] - 0s 9ms/step
[CV] END activation_function=relu, batch_size=32, dropout_rate=0.2, learning_rate=0.001; total time=   4.2s
3/3 [==============================] - 0s 5ms/step
[CV] END activation_function=relu, batch_size=32, dropout_rate=0.2, learning_rate=0.001; total time=   4.7s
3/3 [==============================] - 0s 5ms/step
[CV] END activation_function=relu, batch_size=32, dropout_rate=0.2, learning_rate=0.001; total time=   2.1s
3/3 [==============================] - 0s 4ms/step
[CV] END activation_function=relu, batch_size=32, dropout_rate=0.2, learning_rate=0.0001; total time=   2.1s
3/3 [==============================] - 0s 5ms/step
[CV] END activation_function=relu, batch_size=32, dropout_rate=0.2, learning_rate=0.0001; total time=   2.1s
3/3 [==============================] - 0s 5ms/step
[CV] END activation_function=relu, batch_size=32, dropout_rate=0.2, learning_rate=0.0001; t

RandomizedSearchCV(cv=3, estimator=KerasRandomSearch(),
                   param_distributions={'activation_function': ['relu', 'selu'],
                                        'batch_size': [32, 64],
                                        'dropout_rate': [0.2, 0.5],
                                        'learning_rate': [0.001, 0.0001]},
                   random_state=42, verbose=2)

In [ ]:
# Print the best score and hyperparameter configuration
print("Best score:", random_search.best_score_)
print("Best parameters:", random_search.best_params_)

Best score: 0.8142489711934155
Best parameters: {'learning_rate': 0.0001, 'dropout_rate': 0.2, 'batch_size': 32, 'activation_function': 'relu'}
